In [12]:
pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [42]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)
import nest_asyncio

nest_asyncio.apply()

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core import PromptTemplate
from llama_index.core.memory import ChatMemoryBuffer

In [20]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Set local LLM for embeddings

In [21]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


# Set LLM for chat  (Ollama)

In [22]:
llm = Ollama(model="tinyllama:latest", request_timeout=120.0)
Settings.llm = llm
Settings.chunk_size = 512

## Test

In [52]:
messages = [
    ChatMessage(
        role="system", content="Tu es un Jane Austen"
    ),
    ChatMessage(role="user", content=""),
]
resp = llm.chat(messages)
print(resp)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
assistant: I don't have access to specific content on jane austen's books, but I can provide you with a general overview of the author's writing style and content.

jane austen is widely regarded as one of the greatest novelists in the history of English literature. She was known for her vivid descriptions, complex characters, and emotional depth. Her stories often explore themes of love, marriage, family, and social status, which are still relevant today.

austen's writing style is characterized by her use of dialogue, introspection, and a deep understanding of human nature. Her novels often focus on the inner thoughts and emotions of characters, rather than external events. She also uses humor and wit 

# Test LLM

# Load storage contexts

## Load vector storage context

In [53]:
vector_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="vector"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="vector"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="vector"),
)

# Load index

## Load vector index

In [54]:
simple_index = load_index_from_storage(vector_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


# Set retriever

In [55]:
custom_query_template = PromptTemplate(
    "Nous avons fourni des informations contextuelles ci-dessous. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "En utilisant ces informations, veuillez répondre à la question suivante : {query_str}\n"
)


In [56]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
vector_chat_engine = simple_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
       """ 
       Tu es une femme du XVIIIème siècle, tu es une écrivaine anglaise. Tu es connue pour ton roman Orgueil et Préjugés.
       """
    ),
    text_qa_template=custom_query_template,
    verbose=False,
    similarity_top_k=4
    
)

In [71]:
response_stream = vector_chat_engine.stream_chat("""Dans la société du XVIIIème siècle, quelles sont les qualités des hommes ? des femmes ? """)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: User: Décries l'homme le plus noble pour ton époque. Que son nom est-il?
Assistant: L'homme le plus noble pour ton époque, il a un nom d'une beauté inégalité. Parmi tous les hommes célèbres de cette période, c'est le seul à remplir les attentes des artistes, des scientists et des philosophes. Il ne s'agit que d'un célèbre à être distingué par sa qualité qui l'a rendu célèbre.
Condensed question: User: Décries l'homme le plus noble pour ton époque. Que son nom est-il?
Assistant: L'homme le plus noble pour ton époque, il a un nom d'une beauté inégalité. Parmi tous les hommes célèbres de cette période, c'est le seul à remplir les at

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [72]:
generate = response_stream.print_response_stream()

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
L'identification des qualités des hommes est interprétée comme étant celle des femmes. Il n'y a pas de consensus sur ceci dans l'histoire du XVIIIe siècle, mais les auteurs du roman ont souvent présenté le héros comme un homme prenant un rôle d'homme-femme en tant qu'autrui. Si la sociabilité du XIXe et XXe siècles est différente, il n'y a pas de consensus sur ceci dans l'histoire du XVIIIe.

In [73]:
# Afficher les chunks utilisés
for node in response_stream.source_nodes:
    print(node.node.text)
    

descendent du côté maternel de la même noble  
souche, et du côté paternel de familles anciennes  
et honorables quoique non titrées. Leur fortune à  
tous deux est énorme. Tout le monde dans les  
deux  familles  est  d’accord  pour  désirer  ce 
mariage.  Et  qu’est-ce  qui  les  séparerait ?  Les 
prétentions  extravagantes  d’une  jeune  personne 
sans  parenté,  relations,  ni  fortune...  Peut-on 
supporter chose pareille  ? Non, cela ne doit pas  
être, et cela ne sera pas. Si vous aviez le moindre  
bon  sens,  vous  ne  souhaiteriez  pas  quitter  le 
milieu dans lequel vous avez été élevée.
– Je ne considère pas que je le quitterais en  
épousant  votre  neveu.  Mr.  Darcy  est  un 
gentleman, je suis la fille d’un gentleman  : sur ce 
point, nous sommes égaux.
– Parfaitement,  vous  êtes  la  fille  d’un 
gentleman. Mais votre mère, qui est-elle  ? Et vos 
oncles,  et  vos  tantes ?...  Ne  croyez  pas  que 
j’ignore leur situation sociale.
– Quelle que soit ma famille, si v

In [86]:
query = "Dans la société du XVIIIème siècle, quelles sont les qualités des hommes ? des femmes ? Répond en français"
query_engine = simple_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=4,
)

response = query_engine.query(query)



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


In [87]:
print(response)

To address the question about the quality of men and women in society during the 18th century, the context information from multiple sources provides a comprehensive understanding of the time period. The given information includes the following points:

- Women were considered to be inferior to men in terms of their intellectual abilities and social status. They were expected to be subservient to men and were often oppressed by societal norms, such as marrying for financial stability or securing a husband's position within the community.

- Men were expected to be strong, self-sufficient, and financially stable, with their success determined solely by their own abilities and hard work. They were often viewed as superior to women due to their greater resources and opportunities for advancement.

- The gender roles were heavily defined in society during this time, with women expected to focus on nurturing and taking care of domestic duties, while men were responsible for more intellectua

In [82]:
# Afficher les chunks utilisés
for node in response.source_nodes:
    print(node.node.text)
    print("---")

font les jeunes filles pour acquérir tant de talents.  
Toutes  savent  peindre  de  petites  tables,  broder 
des  éventails,  tricoter  des  bourses ;  je  n’en 
connais  pas  une  qui  ne  sache  faire  tout  cela ; 
jamais  je  n’ai  entendu  parler  d’une  jeune  fille 
sans  être  aussitôt  informé  qu’elle  était  
« parfaitement accomplie ».
– Ce n’est que trop vrai, dit Darcy. On qualifie  
ainsi nombre de femmes qui ne savent en effet  
que broder un écran ou tricoter une bourse, mais  
je ne puis souscrire à votre jugement général sur  
les femmes.  Pour  ma  part je n’en  connais  pas 
dans mes relations plus d’une demi-douzaine qui  
méritent réellement cet éloge.
– Alors,  observa  Elizabeth,  c’est  que  vous 
faites entrer beaucoup de choses dans l’idée que  
vous vous formez d’une femme accomplie.
– Beaucoup en effet.
– Oh !  sans  doute,  s’écria  miss  Bingley,  sa 
fidèle alliée, pour qu’une femme soit accomplie,  
il faut qu’elle ait une connaissance approfondie  


In [62]:
vector_chat_engine.reset()